In [ ]:
# Configure asyncio for Jupyter notebook
# This allows nested event loops which ib_insync needs in a notebook environment
import nest_asyncio
nest_asyncio.apply()

# Phase 1: Contract Resolution Demo

This notebook demonstrates the contract resolution functionality from Phase 1 of the IBKR Gateway project.

**Prerequisites:**
- IBKR Gateway or TWS running on paper trading mode
- Default connection: `127.0.0.1:4002`

## Features Demonstrated:
1. IBKRClient connection management
2. Contract resolution for different security types (STK, ETF, FUT, IND)
3. Futures front-month auto-resolution
4. Contract caching
5. Batch contract resolution

In [ ]:
# Add parent directory to path for imports
import sys
sys.path.insert(0, '..')

from ibkr_core import (
    IBKRClient,
    SymbolSpec,
    resolve_contract,
    resolve_contracts,
    get_contract_cache,
    get_front_month_expiry,
    ContractResolutionError,
    ContractNotFoundError,
)

## 1. Connect to IBKR Gateway

In [ ]:
# Create client and connect
client = IBKRClient(mode="paper")
client.connect(timeout=10)

print(f"Connected: {client.is_connected}")
print(f"Mode: {client.mode}")
print(f"Host: {client.host}:{client.port}")
print(f"Client ID: {client.client_id}")
print(f"Managed accounts: {client.managed_accounts}")

## 2. Resolve Stock Contracts (STK)

In [ ]:
# Resolve AAPL stock
aapl_spec = SymbolSpec(symbol="AAPL", securityType="STK")
aapl_contract = resolve_contract(aapl_spec, client)

print("AAPL Contract:")
print(f"  Symbol: {aapl_contract.symbol}")
print(f"  ConId: {aapl_contract.conId}")
print(f"  SecType: {aapl_contract.secType}")
print(f"  Exchange: {aapl_contract.exchange}")
print(f"  Currency: {aapl_contract.currency}")
print(f"  Primary Exchange: {aapl_contract.primaryExchange}")

In [ ]:
# Resolve MSFT stock
msft_spec = SymbolSpec(symbol="MSFT", securityType="STK")
msft_contract = resolve_contract(msft_spec, client)

print("MSFT Contract:")
print(f"  Symbol: {msft_contract.symbol}")
print(f"  ConId: {msft_contract.conId}")
print(f"  Exchange: {msft_contract.exchange}")
print(f"  Currency: {msft_contract.currency}")

## 3. Resolve ETF Contracts

In [ ]:
# Resolve SPY ETF
spy_spec = SymbolSpec(symbol="SPY", securityType="ETF")
spy_contract = resolve_contract(spy_spec, client)

print("SPY Contract:")
print(f"  Symbol: {spy_contract.symbol}")
print(f"  ConId: {spy_contract.conId}")
print(f"  SecType: {spy_contract.secType}")
print(f"  Exchange: {spy_contract.exchange}")

## 4. Resolve Futures Contracts (FUT) with Auto Front-Month

In [ ]:
# Resolve MES micro e-mini futures (auto front-month)
mes_spec = SymbolSpec(symbol="MES", securityType="FUT")
mes_contract = resolve_contract(mes_spec, client)

print("MES Futures Contract (Front Month):")
print(f"  Symbol: {mes_contract.symbol}")
print(f"  ConId: {mes_contract.conId}")
print(f"  SecType: {mes_contract.secType}")
print(f"  Exchange: {mes_contract.exchange}")
print(f"  Expiry: {mes_contract.lastTradeDateOrContractMonth}")
print(f"  Multiplier: {mes_contract.multiplier}")

In [ ]:
# Resolve ES e-mini futures
es_spec = SymbolSpec(symbol="ES", securityType="FUT")
es_contract = resolve_contract(es_spec, client)

print("ES Futures Contract (Front Month):")
print(f"  Symbol: {es_contract.symbol}")
print(f"  ConId: {es_contract.conId}")
print(f"  Expiry: {es_contract.lastTradeDateOrContractMonth}")
print(f"  Multiplier: {es_contract.multiplier}")

In [ ]:
# Get front month expiry helper
mes_expiry = get_front_month_expiry("MES", client)
es_expiry = get_front_month_expiry("ES", client)

print(f"MES Front Month Expiry: {mes_expiry}")
print(f"ES Front Month Expiry: {es_expiry}")

## 5. Resolve Index Contracts (IND)

In [ ]:
# Resolve SPX index
spx_spec = SymbolSpec(symbol="SPX", securityType="IND")
spx_contract = resolve_contract(spx_spec, client)

print("SPX Index Contract:")
print(f"  Symbol: {spx_contract.symbol}")
print(f"  ConId: {spx_contract.conId}")
print(f"  SecType: {spx_contract.secType}")
print(f"  Exchange: {spx_contract.exchange}")

## 6. Batch Contract Resolution

In [ ]:
# Resolve multiple contracts at once
specs = [
    SymbolSpec(symbol="AAPL", securityType="STK"),
    SymbolSpec(symbol="GOOGL", securityType="STK"),
    SymbolSpec(symbol="AMZN", securityType="STK"),
    SymbolSpec(symbol="TSLA", securityType="STK"),
]

contracts = resolve_contracts(specs, client)

print("Batch Resolution Results:")
print(f"{'Symbol':<10} {'ConId':<12} {'Exchange':<10} {'Currency':<8}")
print("-" * 45)
for symbol, contract in contracts.items():
    print(f"{symbol:<10} {contract.conId:<12} {contract.exchange:<10} {contract.currency:<8}")

## 7. Contract Cache Statistics

In [ ]:
# Check cache statistics
cache = get_contract_cache()
stats = cache.stats

print("Contract Cache Statistics:")
print(f"  Cache size: {stats['size']}")
print(f"  Cache hits: {stats['hits']}")
print(f"  Cache misses: {stats['misses']}")

# Demonstrate cache hit
print("\nResolving AAPL again (should be cache hit)...")
aapl_contract2 = resolve_contract(aapl_spec, client)
stats2 = cache.stats
print(f"  Cache hits after: {stats2['hits']}")

## 8. Error Handling

In [ ]:
# Try to resolve an invalid symbol
try:
    invalid_spec = SymbolSpec(symbol="INVALIDXYZ123", securityType="STK")
    resolve_contract(invalid_spec, client)
except ContractNotFoundError as e:
    print(f"ContractNotFoundError: {e}")
except ContractResolutionError as e:
    print(f"ContractResolutionError: {e}")

## 9. Disconnect

In [ ]:
# Clean up - disconnect from gateway
client.disconnect()
print(f"Disconnected. Is connected: {client.is_connected}")

## Summary

Phase 1 provides:
- **IBKRClient**: Connection management with paper/live mode support
- **Contract Resolution**: Map SymbolSpec to qualified IBKR contracts
- **Security Type Support**: STK, ETF, FUT, OPT, IND
- **Futures Auto-Resolution**: Automatically selects front-month contract
- **Caching**: Reduces API calls with in-memory contract cache
- **Error Handling**: Structured exceptions for resolution failures